In [ ]:
# Wir starten wieder mit einem frischen Jupyter notebook und importieren 
# unsere Pandas Standardbibliothek und natürlich auch geopandas, weil 
# wir ja wieder Kartendaten darstellen wollen und mit Geodataframes arbeiten.

import pandas as pd
import geopandas as gpd

In [ ]:
# Wir wollen jetzt aber noch einen Schritt weitergehen und die Karte ein wenig interaktiver machen.
# Dazu importieren wir die Bibliothe folium. 

import folium

# Die Bibliothek folium enthält ein sehr interessantes Feature mit dem wir uns genauer beschäftigen 
# wollen, das Choropleth. Eine Choropleth-Karte kann verwendet werden, um bestimmte Daten
# auf der Karte durch Einfärbung zu visualisieren.

# Das Ganze zu finden unter Github, nämlich 
# unter https://github.com/python-visualization/folium

# Wir benötigen einen zusätzlichen import-Befehl für das Choropleth Element

from folium import Choropleth

In [ ]:
# Was kann folium? folium bietet uns eine interaktive Karte, die wir bewegen und zoomen können.
# Das hat erstmal garnichts mit unseren Covid19-Daten zu tun.

# Definieren wir einmal eine Karte mit der Sicht auf Deutschland. Dazu wählen wir 
# einen zentralen Punkt von Deutschland. Wikipedia liefert uns hier als 
# ungefähren Wert 51° Nord, 10° Ost, das reicht für unsere Zwecke hinsichtlich der Genauigkeit.
# Wir wählen noch die Zoomstufe 6 als Startwert (habe ich vorher ausprobiert) und los geht es.
# Das Ergebnis des Funktionsaufrufs (eine Karte) speichern wir dann in map1, weil wir 
# sie später noch brauchen werden:

map1 = folium.Map(location=[51,10], tiles='openstreetmap', zoom_start=6)

In [ ]:
map1

# Das ist doch schon mal echt klasse. Skalierbar, verschiebbar und perfekte Kartendaten!
# Das kartenmaterial kann man sogar auswählen, indem man im Parameter 'tiles' die Datenquelle angibt.
# Wir verwenden hier die Kartendaten von openstreetmap.org Projekt.

# Wichtig: Die Verwendung dieses Kartenmaterials geht ok, wenn man auf das openstreetmap.org
# Projekt hinweist, also „© OpenStreetMap-Mitwirkende“.
# Diese Daten sind unter der Open-Database-Lizenz verfügbar. 
# Also bei jeder Nutzung entsprechendauf www.openstreetmap.org/copyright verlinken.

# Wir sehen unten in der Karte auch noch einen weitern Hinweis 
# auf eine weitere Open Source Bibliothek, mit dem Namen "leaflet":
# Auch hier immer auf den Ursprung hinweisen, am besten mit dem Link auf das Original: 
# https://github.com/Leaflet/Leaflet


In [ ]:
# So schön diese Karte auch ist, wir wollen ja einen Zusammenhang mit den RKI Covid Daten herstellen.

# In einem ersten Schritt laden wir wieder die Landkreisdaten vom RKI.
# Wer die Daten noch nicht runtergeladen hat, oder wer die tagesaktuellen Daten haben möchte geht zu
# https://npgeo-corona-npgeo-de.hub.arcgis.com/, gibt "Landkreise" in der Suchmaske ein
# und sucht dort nach den RKI Landkreisen. Dann Download und dann das Shape File laden.
# Genaueres dazu findet ihr im 3. Tutorial.

# Aber zurück zu den Daten. Das war ja ein Shapefile und wir laden das mit dem Befehl 
# gpd.read_file(). Als Ergebnis erhalten wir den Geodatenframe lk_full.

lk_full=gpd.read_file('landkreise/RKI_Corona_Landkreise.shp')

In [ ]:
# lk_full enthält nun wieder alle Informationen über Stadt-/Landkriese aber auch inklusive der Geo-Daten.
# Also einfach lk_full.head() aufrufen zum anschauen der ersten 5 Zeilen der Tabelle:

lk_full.head(3)

# Die Geodaten finden wir in der letzten Spalte.

In [ ]:
# Zur Weiterverarbeitung basteln wir uns einen Geodatenframe, der nur die Geometrieinformationen 
# zum jeweileigen Stad- oder Landkreis enthält:
  
lkgeo=lk_full[['county', 'geometry']].set_index('county')
lkgeo['county']=lkgeo.index


In [ ]:
# Wir brauchen nun noch einen weiteren Datenframe, der die Inzidenzdaten enthält.
# Hier reicht ein pandas Dataframe aus, das muss keine Geodataframe sein:

inz_data=lk_full[['county', 'cases7_p_1']].set_index('county')
inz_data['county']=inz_data.index


In [ ]:
# Aus dem dritten Tutorial wissen wir, dass die Inidenzdaten so einach nicht verarbeitet werden können,
# da sie Kommas statt Dezimalpunkte enthalten. Also kurz Kommas mit Dezimalpunkten ersetzen und als float
# abspeichern:

inz_data['Inzidenz'] = inz_data['cases7_p_1'].str.replace(',','.').astype(float)

In [ ]:
# Die Daten mit den Kommas brauchen wir nicht mehr und wir löschen sie einfach.
# Das geht einfach mit dem drop Befehl.
# Dem müssen wir aber mitgeben, dass wir eine Spalte meinen, also axis=1 und dass wir das 
# ganze permanent in dem Dataframe machen wollen. Also setzen wir wieder Inplace=True

inz_data.drop('cases7_p_1', axis=1, inplace=True)

In [ ]:
# jetzt kommen wir zum eigentlich spannenden Feature von folium.
# Dem dynamischen Overlay, dem sog. Chloropleth:
# Aus der Doku unter https://python-visualization.github.io/folium/
# findet man folgende Konstrution:

cp=Choropleth(
    geo_data=lkgeo.__geo_interface__,  # Link auf die Geo Daten (*)
    name="Inzidenz",                   # Name des Chloropleth, sehen wir so später in der Karte
    data=inz_data,                     # Link zu unserem Inzidenz-Datenframe
    columns=["county", "Inzidenz"],    # Welche Infos wollen wir anzeigen?
    bins=[0, 100, 250, 500, 750, 1000, 2000], # Farbabstufung der Inzidenzen
    key_on="feature.id",               # Interner Link zwischen Geodaten und Inzidenzdaten
    fill_color='YlGnBu',               # Verwendete Farbtabelle
    legend_name='Inzidenz'             # Name, der auf der Legende erscheint
).add_to(map1)        # Am Ende steht noch, dass das Choropleth-Overlay 
                      # mit unserer Karte Map1 verbunden werden soll.


# (*) Das ist jetzt komplex: folium braucht als Geodaten eine Datenstruktur im json Format
# Damit wir es aber nicht so schwer haben konvertiert der Eintrag .__geo_interface__
# unseren Geodatenframe intern um in eine json Datenstruktur    

In [ ]:
# So, wie sieht unsere Karte nun aus? Rufen wir sie wieder auf:

map1

In [ ]:
# Für eine weitere Eigenschaft der folium Biblothek müssen wir unsere Geo Informationen erweitern.
# Das ist auch etwas komplexer, da folium prinzipiell eine json Datenstruktur als Geodaten erwartet.
# Diese hat bestimmte Eigenschaften und die müssen nun um die Eigenschaft Inzidenz erweitert werden.
# Das Thema json ist für dieses Tutorial aber zu umfangreich, deswegen wollen wir das hier nur kurz anschneiden.
# Als neue Eigenschaft also "Property" im den json Geodaten wollen wir nun die Inzidenz einfügen.
# Dazu laufen wir durch alle Einräge der json daten.


for s in cp.geojson.data['features']:
    s['properties']['Inzidenz'] = inz_data.loc[s['id'], 'Inzidenz']

In [ ]:
# Jetzt fügen wir unserem Choropleth noch eine Infobox, einen sog. Tooltip hinzu,
# in dem die Infos zu Landkreisnamen und Ìnzidenz stehen:

folium.GeoJsonTooltip(['county', 'Inzidenz']).add_to(cp.geojson)

In [ ]:
# Karte wieder aufrufen, kurz warten un hier ist nun jeder Landkreis
# mit einem Tooltip versehen. Das Ganze voll dynamisch!

map1

In [ ]:
# Last but not least fügen wir noch eine Schaltfläche hinzu, um den Choropleth an- und auszuschalten.
# Das kennt man aus anderen Kartenprogrammen als Layer.

folium.LayerControl().add_to(m_1)

In [ ]:
# Karte wieder aufrufen

m_1

In [ ]:
# Das obige kann man natürlich auch alles in einen Schritt zusammenfassen, um schnell an die
# gewünschte Kartendarstellung zu kommen.